In [1]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from tqdm import tqdm
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches

/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class ImagesDataProcess(Dataset):
    def __init__(self,csv_file,root_dir,tranform=None):
        self.annotation=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.tranform=tranform
        
    def __len__(self):
        return len(self.annotation)
    
    def __getitem__(self,index):
        img_path=os.path.join(self.root_dir,self.annotation.iloc[index,0])
        image=io.imread(img_path)
        image=resize(image,(250,250),anti_aliasing=True)
        y_label=torch.tensor(int(self.annotation.iloc[index,1]))
        
        if self.tranform:
            image=self.tranform(image)
        return (image,y_label)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
# Hyperparameters
in_channel = 3
num_classes = 3
learning_rate = 1e-3
batch_size = 32
num_epochs = 100
#root_dir_path='../input/cat-dog-images-for-classification/cat_dog'
root_dir_path="dataset"
#csv_file_path='../input/cat-dog-imagEes-for-classification/cat_dog.csv'
csv_file_path="us_GA.csv"

In [5]:
dataset=ImagesDataProcess(
    csv_file=csv_file_path,
    root_dir=root_dir_path,
    tranform=transforms.ToTensor()
)


In [6]:
train_set, test_set = torch.utils.data.random_split(dataset, [468, 117])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)


In [7]:
# Model
model = torchvision.models.googlenet(pretrained=True)
model.fc=nn.Sequential(
    nn.Linear(in_features=1024,out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=3,bias=True)
) 
model.to(device)
model

/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [8]:
from torchsummary import summary

print(summary(model, (3, 250, 250),verbose=0))

Layer (type:depth-idx)                   Output Shape              Param #
├─BasicConv2d: 1-1                       [-1, 64, 125, 125]        --
|    └─Conv2d: 2-1                       [-1, 64, 125, 125]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 125, 125]        128
├─MaxPool2d: 1-2                         [-1, 64, 62, 62]          --
├─BasicConv2d: 1-3                       [-1, 64, 62, 62]          --
|    └─Conv2d: 2-3                       [-1, 64, 62, 62]          4,096
|    └─BatchNorm2d: 2-4                  [-1, 64, 62, 62]          128
├─BasicConv2d: 1-4                       [-1, 192, 62, 62]         --
|    └─Conv2d: 2-5                       [-1, 192, 62, 62]         110,592
|    └─BatchNorm2d: 2-6                  [-1, 192, 62, 62]         384
├─MaxPool2d: 1-5                         [-1, 192, 31, 31]         --
├─Inception: 1-6                         [-1, 256, 31, 31]         --
|    └─BasicConv2d: 2-7                  [-1, 64, 31, 31]          --
|

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in tqdm(enumerate(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device,dtype=torch.float)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")


15it [00:39,  2.63s/it]


Cost at epoch 0 is 0.8936998089154561


15it [00:37,  2.48s/it]


Cost at epoch 1 is 0.6309471110502879


15it [00:36,  2.44s/it]


...

Cost at epoch 97 is 0.0012955616368950965


15it [00:39,  2.65s/it]


Cost at epoch 98 is 0.002889259150348759


15it [00:36,  2.44s/it]

Cost at epoch 99 is 0.0008061177594451389


In [11]:
# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device,dtype=torch.float)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


In [12]:
print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Checking accuracy on Training Set


100%|██████████| 15/15 [00:35<00:00,  2.39s/it]


Got 468 / 468 with accuracy 100.00
Checking accuracy on Test Set


100%|██████████| 4/4 [00:08<00:00,  2.22s/it]

Got 103 / 117 with accuracy 88.03


In [13]:
Path="./model_best.pth"
torch.save(model, Path)

In [ ]:
./#test0=test_set[(test_set[y_label]==0)]
#test1== test_loader[y_label]=1
#test2== test_loader[y_label]=2

In [14]:
idx_0 = [i for (i, (_, y_label)) in enumerate(test_set) if y_label == torch.tensor(0)]
idx_1 = [i for (i, (_, y_label)) in enumerate(test_set) if y_label == torch.tensor(1)]
idx_2 = [i for (i, (_, y_label)) in enumerate(test_set) if y_label == torch.tensor(2)]

In [15]:
import torch.utils.data as data_utils
test_set0 = data_utils.Subset(test_set, idx_0)
test_set1 = data_utils.Subset(test_set, idx_1)
test_set2 = data_utils.Subset(test_set, idx_2)

In [16]:
test_loader0 = DataLoader(dataset=test_set0, batch_size=batch_size, shuffle=True)
test_loader1 = DataLoader(dataset=test_set1, batch_size=batch_size, shuffle=True)
test_loader2 = DataLoader(dataset=test_set2, batch_size=batch_size, shuffle=True)

In [17]:
check_accuracy(test_loader0, model)
check_accuracy(test_loader1, model)
check_accuracy(test_loader2, model)

100%|██████████| 2/2 [00:02<00:00,  1.43s/it]


Got 30 / 37 with accuracy 81.08


100%|██████████| 3/3 [00:04<00:00,  1.66s/it]


Got 64 / 65 with accuracy 98.46


100%|██████████| 1/1 [00:01<00:00,  1.15s/it]

Got 9 / 15 with accuracy 60.00


In [18]:
from PIL import ImageEnhance
import os
import numpy as np
from PIL import Image
def brightnessEnhancement(root_path,img_name):#亮度增强
    image = Image.open(os.path.join(root_path, img_name))
    enh_bri = ImageEnhance.Brightness(image)
    # brightness = 1.1+0.4*np.random.random()#取值范围1.1-1.5
    brightness = 1.5
    image_brightened = enh_bri.enhance(brightness)
    return image_brightened

def contrastEnhancement(root_path, img_name):  # 对比度增强
    image = Image.open(os.path.join(root_path, img_name))
    enh_con = ImageEnhance.Contrast(image)
    # contrast = 1.1+0.4*np.random.random()#取值范围1.1-1.5
    contrast = 1.5
    image_contrasted = enh_con.enhance(contrast)
    return image_contrasted

def rotation(root_path, img_name):
    img = Image.open(os.path.join(root_path, img_name))
    random_angle = np.random.randint(-2, 2)*90
    if random_angle==0:
     rotation_img = img.rotate(-90) #旋转角度
    else:
        rotation_img = img.rotate( random_angle)  # 旋转角度
    # rotation_img.save(os.path.join(root_path,img_name.split('.')[0] + '_rotation.jpg'))
    return rotation_img

def flip(root_path,img_name):   #翻转图像
    img = Image.open(os.path.join(root_path, img_name))
    filp_img = img.transpose(Image.FLIP_LEFT_RIGHT)
    # filp_img.save(os.path.join(root_path,img_name.split('.')[0] + '_flip.jpg'))
    return filp_img

def createImage(imageDir,saveDir):
   i=0
   for name in os.listdir(imageDir):
      i=i+1
      saveName="cesun"+str(i)+".png"
      saveImage=contrastEnhancement(imageDir,name)
      saveImage.save(os.path.join(saveDir,saveName))
      saveName1 = "flip" + str(i) + ".png"
      saveImage1 = flip(imageDir,name)
      saveImage1.save(os.path.join(saveDir, saveName1))
      saveName2 = "brightnessE" + str(i) + ".png"
      saveImage2 = brightnessEnhancement(imageDir, name)
      saveImage2.save(os.path.join(saveDir, saveName2))
      saveName3 = "rotate" + str(i) + ".png"
      saveImage = rotation(imageDir, name)
      saveImage.save(os.path.join(saveDir, saveName3))

In [21]:
#import createImage
imageDir="dataset" #要改变的图片的路径文件夹
saveDir="dataset_aug"  #数据增强生成图片的路径文件夹

In [22]:
createImage(imageDir,saveDir)

/home/amax/.conda/envs/pyr/lib/python3.7/site-packages/ipykernel_launcher.py:33: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
